In [2]:
import json
import pandas as pd
import requests

* Contamos con  100  renglones y 22  columnas.
* Todos los nombres de columnas están en mayúsculas. --> Si negocio lo solicita, podrían dejarse todos en minúsculas.
* Tenemos 3 variables que tienen solo un valor, pero una de ellas es nombre de columna vacío y contenido vacío, por lo tanto habría que quitarla.
* Tenemos 3 variables que se están tomando como de tipo String (o texto), pero deben estar en formato de fecha o timestamp.
* En el conteo de NA´s nos sale que no hay ninguno. Sin embargo, al revisar los datos a detalle podemos ver que estos nulos están representados con vacío (''), por lo que esto deberá ser cambiado en la etapa de Transform.
* Dos variables contienen ligas a páginas de internet. Podríamos suponer, con fines de hacer una transformación más, que se quiere mantener esta información privada, para que no todos tuvieran acceso a ella.
* La columna `_id` no necesariamente debería llegar al usuario final, pues es un indicador más para quien hace la extracción de los datos que de quien hace analítica de datos.
* Las columnas de tipo texto tienen acentos, caracteres especiales, Altas y Bajas.
* Si negocio lo solicitara, podría requerirse quitar las columnas con una sola variable, Este tipo de información debe ser validada con el área de negocio.
* En el caso de la variable `CODIGOPOSTAL` podríamos sacar dos conclusiones: 
    1. el valor que se utiliza para números nulos es el 0
    2. podría parecer que debe ser numérica, pero en realidad debe ser de tipo String, pues no tiene sentido usarla como numérica.
* En cuanto a la variable `FECHALIMITE`, si el plazo fuera uno fijo establecido se podría inputar desde la fecha de solicitud, sumandole los días que se tienen como fecha límite.
* En la variable `TEXTORESPUESTA`, se tienen algunos datos duplicados que parecería que no lo están. Para esto, podría ser necesario hacer una modificación manualmente. 

![image.png](../../imagenes/eda_images/eda_img_01.jpg)


Toda esta información deberá ser utilizada en la etapa de transform para limpiar y estandarizar los datos.

# Leyendo los datos
Utilizaremos los datos de las solicitudes al INAI, por ser un dataset con más tipos de datos y otro posible tipo de complicaciones.

In [3]:
data_raw = requests.get('https://api.datos.gob.mx/v1/inai.solicitudes')
data = data_raw.text
json_data = json.loads(data)
pandas_data = pd.json_normalize(json_data['results'])
pandas_data.head(5)

,_id,FOLIO,FECHASOLICITUD,DEPENDENCIA,ESTATUS,MEDIOENTRADA,TIPOSOLICITUD,DESCRIPCIONSOLICITUD,OTROSDATOS,ARCHIVOADJUNTOSOLICITUD,...,RESPUESTA,TEXTORESPUESTA,ARCHIVORESPUESTA,FECHARESPUESTA,PAIS,ESTADO,MUNICIPIO,CODIGOPOSTAL,SECTOR,
0,56eb2f18d288290100392fb6,'0673800000103,12/06/2003 07:42,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,,...,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,,02/07/2003,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público,
1,56eb2f18d288290100392fb7,'0673800000203,12/06/2003 07:42,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,,...,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,,02/07/2003,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público,
2,56eb2f18d288290100392fb8,'0608400000103,12/06/2003 07:47,AGROASEMEX S.A.,Terminada,Electrónica,Información Pública,Solicito una copia simple del Expediente del C...,José Octavio López Presa prestó servicios prof...,,...,No se dará trámite a la solicitud,Su solicitud expresa manifestada en su respue...,,12/08/2003,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público,
3,56eb2f18d288290100392fb9,'0000700000103,12/06/2003 08:10,SECRETARÍA DE LA DEFENSA NACIONAL,Terminada,Electrónica,Información Pública,Cuál es el armamento y equipo bélico con el qu...,,,...,La información está disponible públicamente,Direcci&oacute;n de Internet en donde se encue...,,16/06/2003,México,Distrito Federal,TLALPAN,14141,Defensa Nacional,
4,56eb2f18d288290100392fba,'0000700000203,12/06/2003 08:11,SECRETARÍA DE LA DEFENSA NACIONAL,Terminada,Electrónica,Información Pública,Cuál es el armamento y equipo bélico con el qu...,,,...,La información está disponible públicamente,Direcci&oacute;n de Internet en donde se encue...,,16/06/2003,México,Distrito Federal,TLALPAN,14141,Defensa Nacional,


Tal como lo vimos en la clase práctica de EDA, tenemos empezaremos por seleccionar las columnas que si contienen información, eliminando la columna vacía.

In [4]:
pandas_data.drop(columns=[''], inplace =True)
pandas_data.columns

Index(['_id', 'FOLIO', 'FECHASOLICITUD', 'DEPENDENCIA', 'ESTATUS',
       'MEDIOENTRADA', 'TIPOSOLICITUD', 'DESCRIPCIONSOLICITUD', 'OTROSDATOS',
       'ARCHIVOADJUNTOSOLICITUD', 'MEDIOENTREGA', 'FECHALIMITE', 'RESPUESTA',
       'TEXTORESPUESTA', 'ARCHIVORESPUESTA', 'FECHARESPUESTA', 'PAIS',
       'ESTADO', 'MUNICIPIO', 'CODIGOPOSTAL', 'SECTOR'],
      dtype='object')

Ahora, pasaremos todas las columnas a minúsculas.

In [5]:
# pandas_data.columns.lower()
lower_cols = (map(lambda x: x.lower(), pandas_data.columns))
pandas_data.columns = lower_cols
pandas_data.columns

Index(['_id', 'folio', 'fechasolicitud', 'dependencia', 'estatus',
       'medioentrada', 'tiposolicitud', 'descripcionsolicitud', 'otrosdatos',
       'archivoadjuntosolicitud', 'medioentrega', 'fechalimite', 'respuesta',
       'textorespuesta', 'archivorespuesta', 'fecharespuesta', 'pais',
       'estado', 'municipio', 'codigopostal', 'sector'],
      dtype='object')

Mientras revisábamos el código postal en el notebook anterior, encontramos que había dos errores en el código postal, pues había ahí un cero y un 1, mientras que el código postal debe estar compuesto por 5 dígitos.

En vista de esto, estos son datos que podrían ser imputados para obtener un mejor resultado.

Los casos mencionados son los siguientes:

In [6]:
codigo_postal_cero = pandas_data.loc[pandas_data['codigopostal']=='0']
codigo_postal_cero

,_id,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,...,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
64,56eb2f18d288290100392ff6,'2031200000103,12/06/2003 09:46,FONDO NACIONAL PARA EL FOMENTO DE LAS ARTESANÍAS,Terminada,Electrónica,Información Pública,Informacion acerca de la operacion de FONART (...,,https://www.infomex.org.mx/gobiernofederal/mod...,...,10/07/2003,Entrega de información en medio electrónico,En relación a su solicitud 2031200000103 del S...,https://www.infomex.org.mx/gobiernofederal/mod...,08/08/2003,Reino Unido,nottingham,Nottingham,0,Desarrollo Social
90,56eb2f18d288290100393010,'0001700000303,12/06/2003 10:23,PROCURADURÍA GENERAL DE LA REPÚBLICA,Terminada,Electrónica,Información Pública,Se solicita copia de los recibos de luz del ed...,,,...,10/07/2003,Entrega de información en medio electrónico,ANEXO RESPUESTA EN WORD,https://www.infomex.org.mx/gobiernofederal/mod...,04/07/2003,México,Distrito Federal,CUAUHTEMOC,0,Procuraduría General de la República


In [7]:
codigo_postal_uno = pandas_data.loc[pandas_data['codigopostal']=='1']
codigo_postal_uno

,_id,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,...,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
51,56eb2f18d288290100392fe9,'0917900000103,12/06/2003 09:28,ADMINISTRACIÓN PORTUARIA INTEGRAL DE MANZANILL...,Terminada,Electrónica,Información Pública,Deseo saber el número de cruceros que llegaron...,,,...,10/07/2003,Entrega de información en medio electrónico,Durante el Año 2002 arribaron al Puerto de Man...,,10/07/2003,México,Distrito Federal,BENITO JUAREZ,1,Comunicaciones y Transportes


Entonces, estos casos serán inputados manualmente, por ser pocos.

En el caso de Nottinham, se dejará el valor que tiene en la base de datos, solamente para identificar que se trata de un codigo postal fuera del país.

In [8]:
pandas_data.loc[(pandas_data['folio'] =="'0001700000303"),'codigopostal']='06000'
pandas_data.loc[(pandas_data['folio'] =="'0917900000103"),'codigopostal']='03000'

Ahora, se realizará la imputación de los valores nulos (En el dataset aparecen como vacío '')

In [9]:
pandas_data_nans = pandas_data.copy()
pandas_data_nans.head(1)

,_id,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,...,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
0,56eb2f18d288290100392fb6,'0673800000103,12/06/2003 07:42,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,,...,,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,,02/07/2003,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público


In [10]:
# pandas_data_nans = pandas_data_nans.replace('',np.nan)
pandas_data_nans = pandas_data_nans.replace('',pd.NA)

In [11]:
pandas_data_nans['archivoadjuntosolicitud'].unique()

array([<NA>,
       'https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=1816400000103',
       'https://www.infomex.org.mx/gobiernofederal/moduloPublico/MimeGenerator.action?folio=2031200000103'],
      dtype=object)

In [12]:
# ['OTROSDATOS', 'ARCHIVOADJUNTOSOLICITUD','MEDIOENTREGA','FECHALIMITE','RESPUESTA','TEXTORESPUESTA',
#  'ARCHIVORESPUESTA',''] #''


# pandas_data['OTROSDATOS']

La columna _id no será necesaria. Por lo tanto, la eliminaremos

In [13]:
pandas_data_nans = pandas_data_nans.drop(columns = ['_id'])
pandas_data_nans.head(2)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
0,'0673800000103,12/06/2003 07:42,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,<NA>,Copia Simple,<NA>,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,<NA>,02/07/2003,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público
1,'0673800000203,12/06/2003 07:42,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,<NA>,Copia Simple,<NA>,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,<NA>,02/07/2003,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público


In [14]:
# Date columns:
date_col = ['fechasolicitud','fecharespuesta','fechalimite']
for col in date_col:
    pandas_data_nans[col] = pd.to_datetime(pandas_data_nans[col])
    
pandas_data_nans.dtypes

folio                              object
fechasolicitud             datetime64[ns]
dependencia                        object
estatus                            object
medioentrada                       object
tiposolicitud                      object
descripcionsolicitud               object
otrosdatos                         object
archivoadjuntosolicitud            object
medioentrega                       object
fechalimite                datetime64[ns]
respuesta                          object
textorespuesta                     object
archivorespuesta                   object
fecharespuesta             datetime64[ns]
pais                               object
estado                             object
municipio                          object
codigopostal                       object
sector                             object
dtype: object

In [15]:
pandas_data_nans.head(2)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
0,'0673800000103,2003-12-06 07:42:00,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,<NA>,Copia Simple,NaT,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,<NA>,2003-02-07,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público
1,'0673800000203,2003-12-06 07:42:00,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,<NA>,Copia Simple,NaT,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,<NA>,2003-02-07,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público


La columna Folio inicia con una apostrofe, lo cual está mal. Borrarlo.

In [16]:
pandas_data_nans["folio"] = pandas_data_nans["folio"].str.replace("'","")
pandas_data_nans.head(2)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
0,0673800000103,2003-12-06 07:42:00,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,<NA>,Copia Simple,NaT,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,<NA>,2003-02-07,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público
1,0673800000203,2003-12-06 07:42:00,INSTITUTO FEDERAL DE ACCESO A LA INFORMACIÓN Y...,Terminada,Electrónica,Información Pública,Solicito una copia simple del Acta del Pleno d...,Todos los martes el Pleno del IFAI lleva a cab...,<NA>,Copia Simple,NaT,No se dará trámite a la solicitud,Es una solicitud prueba con motivo de la puest...,<NA>,2003-02-07,México,Distrito Federal,MIGUEL HIDALGO,11950,Hacienda y Crédito Público


In [17]:
def StringLowercase(df):
    """
    Función cambiar todos los strings de un dataframe a lowercase
    (columnas y observaciones)
    ==========
    * Args:
         - df: dataframe al que se desea hacer la modificación.
    * Return:
         - df: dataframe modificado
    ==========
    Ejemplo:
        >>df = StringLowercase(df)
    """
    ### Columnas

    DataFrameColumns = df.columns

    for col in DataFrameColumns:
        df.rename(columns={col:col.lower()}, inplace=True)

    ### Observaciones

    filtro = df.dtypes == object
    objects = df.dtypes[filtro]
    StringColumns = list(objects.index)

    for col in StringColumns:
        df[col] = df[col].str.lower()

    return df

def StringAcentos(df):
    """
    Función para eliminar acentos, dieresis y eñes de los strings de un
    dataframe (columnas y observaciones)
    ==========
    * Args:
         - df: dataframe al que se desea hacer la modificación.
    * Return:
         - df: dataframe modificado
    ==========
    Ejemplo:
        >>df = StringAcentos(df)
    """
    ### Columnas

    df.columns = df.columns.str.replace('á', 'a')
    df.columns = df.columns.str.replace('é', 'e')
    df.columns = df.columns.str.replace('í', 'i')
    df.columns = df.columns.str.replace('ó', 'o')
    df.columns = df.columns.str.replace('ú', 'u')
    df.columns = df.columns.str.replace('ü', 'u')
    df.columns = df.columns.str.replace('ñ', 'n')

    ### Observaciones

    filtro = df.dtypes == object
    objects = df.dtypes[filtro]
    StringColumns = list(objects.index)

    for col in StringColumns:
        df[col] = df[col].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

    return df

def StringStrip(df):
    """
    Función para eliminar espacios al inicio y al final de los strings de un
    dataframe (columnas y observaciones)
    ==========
    * Args:
         - df: dataframe al que se desea hacer la modificación.
    * Return:
         - df: dataframe modificado
    ==========
    Ejemplo:
        >>df = StringStrip(df)
    """
    ### Columnas

    df.columns = [col.strip() for col in df.columns]

    ### Observaciones

    filtro = df.dtypes == object
    objects = df.dtypes[filtro]
    StringColumns = list(objects.index)

    for col in StringColumns:
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

    return df

def StringEspacios(df):
    """
    Función para eliminar espacios dobles (o mas) de los strings de un
    dataframe (columnas y observaciones)
    ==========
    * Args:
         - df: dataframe al que se desea hacer la modificación.
    * Return:
         - df: dataframe modificado
    ==========
    Ejemplo:
        >>df = StringEspacios(df)
    """
    ### Columnas

    df.columns = [re.sub(' +', ' ', col) for col in df.columns]

    ### Observaciones

    filtro = df.dtypes == object
    objects = df.dtypes[filtro]
    StringColumns = list(objects.index)

    for col in StringColumns:
        df[col] = df[col].apply(lambda x: re.sub(' +', ' ', x) if isinstance(x, str) else x)

    return df

def EstandarizaFormato(df):
    """
    Función para estandarizar un dataframe: minúsculas, sin espacios en blanco,
    sin signos de puntuación (columnas y observaciones)
    ==========
    * Args:
         - df: dataframe al que se desea hacer la modificación.
    * Return:
         - df: dataframe modificado
    ==========
    Ejemplo:
        >>df = EstandarizaFormato(df)
    """
    ### Quita espacios en columnas
    df.columns = df.columns.str.replace(' ', '_')

    ### Minúsculas
    df = StringLowercase(df)

    ### Acentos
    df = StringAcentos(df)

    ### Quitamos espacios al principio y al final
    df = StringStrip(df)

    ### Quitamos espacios
    df = StringEspacios(df)

    return df

In [18]:
import re
df_final = EstandarizaFormato(pandas_data_nans)
df_final.head(2)

,folio,fechasolicitud,dependencia,estatus,medioentrada,tiposolicitud,descripcionsolicitud,otrosdatos,archivoadjuntosolicitud,medioentrega,fechalimite,respuesta,textorespuesta,archivorespuesta,fecharespuesta,pais,estado,municipio,codigopostal,sector
0,0673800000103,2003-12-06 07:42:00,instituto federal de acceso a la informacion y...,terminada,electronica,informacion publica,solicito una copia simple del acta del pleno d...,todos los martes el pleno del ifai lleva a cab...,<NA>,copia simple,NaT,no se dara tramite a la solicitud,es una solicitud prueba con motivo de la puest...,<NA>,2003-02-07,mexico,distrito federal,miguel hidalgo,11950,hacienda y credito publico
1,0673800000203,2003-12-06 07:42:00,instituto federal de acceso a la informacion y...,terminada,electronica,informacion publica,solicito una copia simple del acta del pleno d...,todos los martes el pleno del ifai lleva a cab...,<NA>,copia simple,NaT,no se dara tramite a la solicitud,es una solicitud prueba con motivo de la puest...,<NA>,2003-02-07,mexico,distrito federal,miguel hidalgo,11950,hacienda y credito publico


Listo, hemos concluído con las trasnformaciones.

In [20]:
# list(df_final.itertuples(index=False, name=None))